In [22]:
import argparse, sys, os, re,copy, json,pandas as pd, numpy as np 

In [23]:
sys.path.append('./')
nowpath = os.getcwd()

In [24]:
jsondic = json.load(open('D:\\cyj\\00_project\\02_algaeAI\\2023\\00_dev\\nier\\backend\\.user\\admin\\.model\\MakeTest\\modelinfo.json'))

In [25]:
jsonlink = jsondic['link']
jsonmodel = jsondic['model']
jsoncolrow = {x[2]:[x[1],x[0]] for x in jsondic['colrow']}

In [26]:
deep_link = copy.deepcopy(jsonlink)

In [27]:
cols = {}
mers = {}

In [28]:
for link in jsonlink:
    if 'input' in link['source']:
        cols[link['source'][-1]] = [link['target']]
        deep_link.remove(link)
    elif 'Merge' in link['source']:
        mers[link['source']]={'row':0,'list_cols':[],'following_sequential':[link['target']]}
        deep_link.remove(link)

In [29]:
jsonlink = deep_link

In [30]:
for key in mers.keys():
    for source in mers[key]['following_sequential']:
        for link in jsondic['link']:
            if source == link['source'] and 'Merge' not in link['target']:
                mers[key]['following_sequential'].append(link['target'])
                deep_link.remove(link)
            elif source == link['source'] and 'Merge' in link['target']:
                deep_link.remove(link)

In [31]:
for link in deep_link:
    if 'Merge' not in link['target']:
        cols[link['id'].split(',')[0].split('e')[-1]].append(link['target'])

In [32]:
for key in cols.keys():
    layerlist = []
    for layer in cols[key]:
        for model in jsonmodel:
            if layer == model['newc']:
                att = ''
                for k,v in model['data'].items():
                    att += k + '=' + str(v) + ',' if isinstance(v, int) or isinstance(v, float) or v in ['True', 'False'] else k + '=' + "'" + v + "'" + ','
                layerlist.append({'type':model['type'],'coord':jsoncolrow[layer],'attributes':att})
    cols[key] = sorted(layerlist, key=lambda x: x['coord'][0])

In [33]:
for key in mers.keys():
    mers[key]['row'] = jsoncolrow[key][0]
    layerlist = []
    for layer in mers[key]['following_sequential']:
        for model in jsonmodel:
            if model['newc'] == key:
                mers[key]['list_cols'] = model['merge']
            if layer == model['newc']:
                att = ''
                for k,v in model['data'].items():
                    att += k + '=' + str(v) + ',' if isinstance(v, int) or isinstance(v, float) or v in ['True', 'False'] else k + '=' + "'" + v + "'" + ','
                layerlist.append({'type':model['type'],'coord':jsoncolrow[layer],'attributes':att})
    mers[key]['following_sequential'] = sorted(layerlist, key=lambda x: x['coord'][0])

In [15]:
list_cols = list(cols.values())

In [16]:
list_mers = list(mers.values())

In [34]:
cols

{'1': [{'type': 'Linear',
   'coord': [1, 1],
   'attributes': 'out_features=1,bias=True,'},
  {'type': 'Linear',
   'coord': [2, 1],
   'attributes': 'out_features=1,bias=True,'}],
 '2': [{'type': 'Linear',
   'coord': [3, 2],
   'attributes': 'out_features=1,bias=True,'},
  {'type': 'Tanh', 'coord': [4, 2], 'attributes': ''},
  {'type': 'Dropout', 'coord': [5, 2], 'attributes': 'p=0.5,inplace=False,'}]}